# Start here if working in Colab

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive

drive.mount(ROOT)           # we mount the google drive at /content/drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set working directory
%cd /content/drive/My Drive/restoration-mapper

/content/drive/My Drive/restoration-mapper


# Start here if working locally

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np



```
`# This is formatted as code`
```

### Import Data

In [0]:
test_x = np.load('data/test-processed/test_x_l2a_processed.npy')
test_y = np.load('data/test-processed/test_y_l2a_processed.npy')
test_length = np.load('data/test-processed/test_length_l2a_processed.npy')

train_x = np.load('data/train-processed/data_x_l2a_processed.npy')
train_y = np.load('data/train-processed/data_y_l2a_processed.npy')
train_length = np.load('data/train-processed/length_l2a_processed.npy')


In [0]:
# Remove unknown x variable
train_x = np.delete(train_x, 14, -1)

# Subset data to first time period
train_x = train_x[:, 0, :, :, :]

### Normalize and Standardize data

In [0]:
# Should x data be normalized or standardized?

# Normalize data [x_norm = (x - min) / (max - min)]
minarray = np.min(train_x[:, :, :, :], axis = (0,1,2))
minarray = np.tile(minarray, 4383*16*16).reshape(4383,16,16,16)

max_min_array = np.max(train_x[:, :, :, :], axis = (0,1,2)) - np.min(train_x[:, :, :, :], axis = (0,1,2))
max_min_array = np.tile(max_min_array, 4383*16*16).reshape(4383,16,16,16)

test = (train_x - minarray) / max_min_array

# Standardize data
## TO DO...

In [63]:
# Check if data is normalized
print("Min value of all variables is 0:", (np.min(test[:,:,:,:], axis=(0,1,2)) == 0).all())
print("Max value of all variables is 1:", (np.max(test[:,:,:,:], axis=(0,1,2)) == 1).all())

Min value of all variables is 0: True
Max value of all variables is 1: True


### Define Model

In [0]:
def train():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # model fitting
    history = model.fit(x_train, 
                        y_train, 
                        epochs=10
    )
    # model fitting
    return history.epoch, history.history['acc'][-1]